In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import time,random;

In [ ]:
experiment_25_file = "../src/build/csv/experiment-25-09-2025.csv"
df_experiment = pd.read_csv(experiment_25_file)
df_experiment["warmup_duration"] = df_experiment["warmup_duration"].apply(lambda x: [float(i) for i in x.split('|')])
df_experiment["repetitions_duration"] = df_experiment["repetitions_duration"].apply(lambda x: [float(i) for i in x.split('|')])
df_experiment['warmup_duration'] = df_experiment['warmup_duration'].apply(lambda x: [] if x == [0.0] else x)
df_exp_ref = df_experiment[df_experiment['version'].str.contains("Reference")]


In [ ]:
df_exp_ref

In [ ]:
def boxplot_benchmark_warmup(df_one_ref,fixed_repetitions=400,version="",ax=None):
    warmup_df = df_one_ref[['warmups','repetitions','warmup_duration']].explode('warmup_duration')
    warmup_df = warmup_df.rename(columns={'warmup_duration': 'duration'})
    warmup_df["type"] = "warmups"
    
    repetitions_df = df_one_ref[['warmups','repetitions','repetitions_duration']].explode('repetitions_duration')
    repetitions_df = repetitions_df.rename(columns={'repetitions_duration': 'duration'})
    repetitions_df["type"] = "repetitions"
    long_df = pd.concat([warmup_df, repetitions_df], ignore_index=True)
    long_df.dropna(inplace=True)
    long_df = long_df[long_df["repetitions"]==fixed_repetitions]
    if ax is None:
        plt.figure(figsize=(8,8))
        sns.boxplot(long_df, x="warmups",y="duration",hue="type",fill=False)
        plt.title(f"Benchmarking of wamup for version {version}")
        plt.xlabel("Number of Warmup executions")
        plt.ylabel("Duration of a kernel (ms)")
    else:
        sns.boxplot(long_df, x="warmups",y="duration",hue="type",fill=False,ax=ax)
        ax.set_title(f"Benchmarking of wamup for version {version}")
        ax.set_xlabel("Number of Warmup executions")
        ax.set_ylabel("Duration of a kernel (ms)")
   

In [ ]:
def boxplot_benchmark_repetitions(df_one_ref,fixed_warmup=5,version="",ax=None):
    repetitions_df = df_one_ref[['warmups','repetitions','repetitions_duration']].explode('repetitions_duration')
    repetitions_df = repetitions_df.rename(columns={'repetitions_duration': 'duration'})
    repetitions_df.dropna(inplace=True)
    repetitions_df = repetitions_df[repetitions_df["warmups"]==fixed_warmup]
    if ax is None:
        plt.figure(figsize=(8,8))
        sns.boxplot(repetitions_df, x="repetitions",y="duration",fill=False)
        plt.title(f"Benchmarking of repetitions for version {version}")
        plt.xlabel("Number of repetitions")
        plt.ylabel("Duration of a kernel (ms)")
    else:
        sns.boxplot(repetitions_df, x="repetitions",y="duration",fill=False,ax=ax)
        ax.set_title(f"Benchmarking of repetitions for version {version}")
        ax.set_xlabel("Number of repetitions")
        ax.set_ylabel("Duration of a kernel (ms)")

In [ ]:
df = df_exp_ref[df_exp_ref["version"]=="ReferenceAccuracy"]
boxplot_benchmark_warmup(df,version="ReferenceAccuracy")

In [ ]:
boxplot_benchmark_repetitions(df,version="ReferenceAccuracy")

In [ ]:
count = len(df_exp_ref["version"].unique())
fig, axes = plt.subplots(2, count, figsize=(5*count, 8))   # width grows with X
plt.tight_layout()   # pad = space (inches) around and between subplots
fig.subplots_adjust(hspace=0.3, wspace=0.4)  # vertical & horizontal padding

curr = 0
for i in df_exp_ref["version"].unique():
    df = df_exp_ref[df_exp_ref["version"]==i]
    boxplot_benchmark_warmup(df,version=i,ax=axes[0][curr])
    boxplot_benchmark_repetitions(df,version=i,ax=axes[1][curr])
    curr+=1
plt.savefig(f"./figs/.svg", format="svg", bbox_inches="tight")